# 第3章: 正規表現

## 20. JSONデータの読み込み
### Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [ ]:
import json
import gzip
import pandas as pd

def read_gz_json(file_name):
    with gzip.open(file_name, 'r') as f:
        data = [json.loads(line) for line in f]
    return pd.DataFrame(data)
read_gz_json('data/jawiki-country.json.gz')

## 21. カテゴリ名を含む行を抽出
### 記事中でカテゴリ名を宣言している行を抽出せよ．



In [ ]:
data = read_gz_json('data/jawiki-country.json.gz')
data[data.text.str.contains("Category:")]

## 22. カテゴリ名の抽出
### 記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [ ]:
data.text.str.extractall('Category:(.+)]]')

## 23. セクション構造Permalink
### 記事中に含まれるセクション名とそのレベル（例えば”== セクション名 ==”なら1）を表示せよ．

In [ ]:
result = pd.DataFrame()
for class_num in range(1,10):
    tag = "".join(["=" for _x in range(class_num + 1)])
    df = data.text.str.extractall(f'{tag} (.+) {tag}')
    if df.empty:
        break
    df["class"] = class_num
    result = pd.concat([result,df])
result

## 24. ファイル参照の抽出
### 記事から参照されているメディアファイルをすべて抜き出せ．

In [ ]:
data.text.str.extractall('ファイル:(.+?)\|')

## 25. テンプレートの抽出Permalink
### 記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [ ]:
import re
result = {}
for _index,row in data.iterrows():
    title = row.title
    text = row.text
    dic = {}
    info = False
    for sentense in text.split("\n"):
        if info:
            find_list = re.findall('\|(.+?)\s*=\s*(.+)',sentense)
            if len(find_list) == 0:
                break
            for tup in find_list:
                dic[tup[0]] = tup[1]
        if re.search("{{基礎情報",sentense):
            info = True
    result[title] = dic
result

## 26. 強調マークアップの除去
### 25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: マークアップ早見表）．

In [ ]:
def delete_emphasis(dic:dict) -> dict:
    for country in dic.keys():
        for key in dic[country].keys():
            dic[country][key] = dic[country][key].replace("'''","")
    return dic
result = delete_emphasis(result)
result

## 27. 内部リンクの除去
### 26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: マークアップ早見表）．



In [ ]:
def check_inner(text:str) -> bool:
    file_search = re.search("ファイル:(.+?)\|",text)
    if file_search is not None:
        return False
    file_search = re.search("File:(.+?)\|",text)
    if file_search is not None:
        return False
    return True


def delete_innerlink(dic:dict) -> dict:
    for country in dic.keys():
        for key in dic[country].keys():
            inner_link = re.findall("\[\[(.+?)\]\]",dic[country][key])
            if len(inner_link) == 0:
                continue
            new_text = []
            for item in inner_link:
                if not check_inner(item):
                    new_text.append(f"[[{item}]]")
                else:
                    new_text.append(item.split("|")[-1])
            dic[country][key] = ",".join(new_text)
    return dic
result = delete_innerlink(result)
result

## 28. MediaWikiマークアップの除去Permalink
### 27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

In [ ]:
def delete_markup(dic:dict) -> dict:
    for country in dic.keys():
        for key in dic[country].keys():
            dic[country][key] = re.sub("\{\{(.+?)\}\}","\\1",dic[country][key])        
    return dic
result = delete_markup(result)
result

## 29. 国旗画像のURLを取得する
### テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: MediaWiki APIのimageinfoを呼び出して，ファイル参照をURLに変換すればよい）

In [ ]:
import json
import requests
def get_flagimage(country_dict:dict)->dict:
    if '国旗画像' not in country_dict:
        return {}
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "prop": "imageinfo",
        "titles": f"File:{country_dict['国旗画像']}",
        "iiprop":"url"
    }
    r = requests.get(url=url, params=params)
    request_result = json.loads(r.text)
    image_dict = {}
    for page in request_result['query']['pages'].values():
        image_dict[page["title"]] = [image['url'] for image in page['imageinfo']]
    return image_dict

flag_dict = {}
for country in result.keys():
    flag_dict[country] = get_flagimage(result[country])
flag_dict